# Dataprep - Electronics and TestData

Data here:  http://jmcauley.ucsd.edu/data/amazon/links.html
<br>Project Notes: https://drive.google.com/file/d/1gQqMqMfF-JMjW0ihN5n16u6YzxeUzi6v/viewm

### About this workbook:

The purpose of this workbook is to clean and prepare the data. In summary, we will:
0. Convert Datatypes: Our initial aim was to concert to smaller datatypes in order to work more efficiently. 
1. Remove reviews with duplicate reviewText (keeping only one)
2. Remove reviews that have received less than 50 votes regarding their helpfulness
3. Assign a value of 1.0 (for helpful) to any review which 75% or more of voters have assessed as "Helpful". Any review with less than a 75% rating is assigned a value of 0.0 for "Not Helpful"
4. Create a balanced dataset: Step 3 will result in unbalanced data, where the majority of all reviews will be rated "helpful". To account for such an unbalanced dataset and avoid training the classifier to simply pick the more frequent class, initial experiments will train on a balanced dataset where 50% of the data is "helpful" and 50% is "not helpful". Since the majority of our data is rated "helpful"
5. Initial experiments also use the .from_folder method for TextBlock, and therefore require pre-populated "train" and "test" folders.Populating the folders is done in Workbook electronics_data_prep2.

In [1]:
import time
import pandas as pd

start = time.time()

start = time.time()
df = pd.read_json('../Review_Data/reviews_Electronics_5.json.gz', lines=True, compression="gzip")

end = time.time()
elapsed = end - start
print(elapsed)

78.70409917831421


### Looking at the Dataframe

In [ ]:
df.head()

I like what you guys have done to validate that our "helpful" column really does contain a list as expected of length 2. For reference, we expect [a,b] where a is the number of people who voted "yes, this is a helpful review"  and b is the number of people who voted in total. a-b are the curmudgeons who didn't like the review.

In [ ]:
df["helpful"].apply(lambda x: len(x)).value_counts()

In [ ]:
df["overall"].value_counts()

It appears that most people give fairly high ratings: The 5.0 category has the most values. Let us drop all columns we will not need.

In [ ]:
df.drop(columns=["reviewerID", "asin", "reviewerName", "unixReviewTime", "reviewTime"], inplace=True)

### Converting Datatypes
As recommended by Gaurav, let's see if we can't cast to smaller datatypes. First, let's see what we've got.

In [ ]:
df.dtypes

What is object? Shouldn't the datatype for "reviewText" and "summary" be a string?
Let's check again - maybe it's strings and something else

In [ ]:
df['reviewText'].apply(lambda x: type(x))

Looks like strings, but maybe there's something else hiding? Let's get a summary using value_counts()

In [ ]:
df['reviewText'].apply(lambda x: type(x)).value_counts()

In [ ]:
df['summary'].apply(lambda x: type(x)).value_counts()

So for both columns, there really are just strings but pandas says they're objects. Let's cast them as strings.

In [ ]:
df = df.astype({'overall':'float32','reviewText':'str', 'summary':'str'})

I don't think there's an "inplace" parameter here, so I just assigned it to "df" again.

Huh. If I run dtype on that dataframe, those columns still say object, not string. Oh well. A mystery to be left for another day.

### Adding columns for helpful votes, total votes, and the ratio
We want to make three columns: "num_helpful_votes", "num_total_votes", "helpful_ratio".
The first two columns should be ints, but I'm casting them to floats because they're the numerator and denominator for the division that will be saved in the third column. That way I ensure that third column contains floats. 
I'll just cast the first two back to ints later.

I think Dewsey does the same thing in a beautiful squish function. His function saves you from having to ensure that the first two are floats, and then recast as ints, and from having to use fillna.

In [ ]:
start = time.time()

df["num_helpful_votes"] = df["helpful"].apply(lambda x: float(x[0]))
df["num_total_votes"] = df["helpful"].apply(lambda x: float(x[1]))
df["helpful_ratio"] = df["num_helpful_votes"] / df["num_total_votes"]

end = time.time()
elapsed = end - start
print(elapsed)

In [ ]:
df = df.astype({'num_helpful_votes': 'int32', 'num_total_votes': 'int32'})

In [ ]:
df.dtypes

In [ ]:
df.head()

Let's replace the NaN in helpful_ratio with zeroes

In [ ]:
df['helpful_ratio'].fillna(0, inplace=True)

In [ ]:
df.head()

### Dropping duplicates and separating out the LM dataset and the Classification dataset

#### Duplicates

There are duplicate reviews, likely by scammers. Maybe they can be leveraged somehow later, but for now, we're removing them.

#### Language Model Dataset versus Classification Dataset
Some reviews have too little "helpfulness" votes to be of value for the classification task, but they can be useful for the Language Model fine tuning. 
1. threshhold for determining whether a review should be included in the classification task
The literature used 50. Let's try this. Depending on how much that leaves us, we might reduce it to something smaller.
2. LM Dataset: We theoretically could use all reviews (minus the duplicates) to finetune the language model. However, that would be more than 1 million reviews which is too many because:
<br>a) The fine-tuning of the imdb language model only needed 100K reviews and got good results
<br>b) I tried it twice and the kernel died each time. 

<br>So in summary, it is not necessary for good results and it is for us not practical.

In [ ]:
df_uniq = df.drop_duplicates(subset="reviewText", keep = "first") #we could use inplace=True but I want to keep the old data
#frame around

The original idea was to use as many reviews as possible to finetine the Language Model, and planned to use df_uniq for this since it contained all reviews (minus the duplicates). That turned out to be not feasible (and crashed my kernel several times). However, filtering out all reviews with less 50 helpfulness votes yields us a smaller dataframe of 19K reviews, which proved useful for LM finetuning.

In [ ]:
df_uniq.head()

Let's filter for the rows where the num_total_votes is 50 or more, and see how many there are

In [ ]:
df_uniq[df_uniq['num_total_votes'] > 49 ].shape

Ok, 19,009. Let's see how far that takes us - let's save it.

In [ ]:
df_uniq_cl = df_uniq[df_uniq['num_total_votes'] > 49 ]

In [ ]:
df_uniq_cl.head()

#### Let's bin and balance

In [ ]:
bins = [0.0, 0.75, 1.0]
df_uniq_cl["is_helpful"] = pd.cut(df_uniq_cl["helpful_ratio"], bins, labels=["0", "1"])

In [ ]:
df_uniq_cl.head()

In [ ]:
df_uniq_cl["is_helpful"].value_counts()

#### so we need to redo the below a little differently:
I need: 
1. append the two below so I make dataframe balanced
2. balanced dataframe split into test and train
3. In another workbook, I can use the "test" and "train" dataframes to populate the respective test and train folders, which can then be used in training the LM and Classifier (TextBlock.from_folder method for example requires this pre-populated folder)

Sampling rom "helpful" in order to achieve an equal number of helpful and unhelpful votes

In [ ]:
all_unhelpful = df_uniq_cl.loc[df_uniq_cl["is_helpful"] == "0"]

In [ ]:
all_unhelpful['is_helpful'].value_counts()

In [ ]:
all_helpful = df_uniq_cl.loc[df_uniq_cl["is_helpful"] == "1"].sample(n=2010, random_state=42, axis=0)

In [ ]:
all_helpful["is_helpful"].value_counts()

Ok, the helpful/unhelpful dataframes look good. Let's append them.

In [ ]:
balanced_df = all_helpful.append(all_unhelpful)

In [ ]:
balanced_df['is_helpful'].value_counts()

The balanced dataframe also looks good. We have equal counts of "1" and "0" in the 'is_helpful' column. Now, let's divide this into test and training data. Borrowing' Gaurav's approach here again. Note: I bet there is a splitter method in fastai, but so far, I've seen that splitter method integrated into Data loader classes, so I don't know where to find it alone at the moment. 

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(balanced_df, test_size=0.20, stratify=balanced_df["is_helpful"], random_state=42)

train_df.shape, test_df.shape

In [ ]:
train_df.to_csv("electronics_5_train.csv", header=True, index=False)
test_df.to_csv("electronics_5_test.csv", header=True, index=False)

We could also export the full helpful/unhelpful ones, but no need.We could also export the entire set of training/test data, uniqued, but not balanced (df_uniq_cl)

In [ ]:
df_uniq_cl.to_csv("electronics_all_classifier_data_unbalanced_uniq.csv", index=False)

Before exporting the LM dataset, let's remove the columns we don't need for training the LM.

In [ ]:
df_uniq_LM = df_uniq.drop(["helpful","overall",'num_helpful_votes','num_total_votes','helpful_ratio'], 1)

In [ ]:
df_uniq_LM.head()

In [ ]:
df_uniq_LM.to_csv("electronics_for_LM_uniq.csv", index=False)